In [1]:
# myenv(3.12.6)

rectified_text_2023_2.txt<br>
RecursiveCharacterTextSplitter<br>
jeonseonjin/embedding_BAAI-bge-m3<br>
FAISS<br>
Retrieval 과정에서 중복제거<br>
beomi/gemma-ko-2b<br>

In [2]:
#%pip install huggingface_hub langchain langchain-community faiss-cpu sentence-transformers torch accelerate

In [3]:
#Use 3.12.4
import os
from huggingface_hub import login
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.docstore.document import Document


key_path = '/Users/jaesolshin/key/HF_TOKEN.txt'
os.environ["HF_TOKEN"] = open(key_path, 'r', encoding='utf-8').read()
login(os.environ["HF_TOKEN"], add_to_git_credential=True)

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/jaesolshin/.cache/huggingface/token
Login successful


# Document Loaders

In [4]:
path = '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023_2.txt'
loader = TextLoader(path)
documents = loader.load()

In [5]:
print(f'문서개수: {len(documents)}')
print(f'첫번째 문서의 내용: {documents[0].page_content}')
print(f'첫번째 문서의 메타데이터: {documents[0].metadata}')

문서개수: 1
첫번째 문서의 내용: 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다. 초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하였다. 일부 업권의 건전성 우려, 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었다. 자산운용사 수신은 주가 상승, 대내외 긴축기조 완화 기대 등으로 주식형 및 채권형 펀드로 자금이 유입되면서 증가 폭이 확대되었다. 2023년 중 기업대출은 은행과 비은행 예금 취급 기관 모두 전년 대비 증가 폭이 축소되었다. 특히 비은행 예금 취급 기관 기업대출은 부동산 경기 부진 등으로 건설·부동산업 관련 대출 수요가 크게 둔화되면서 증가 규모가 전년 대비 큰 폭으로 축소되었다. 한편, 은행 기업대출의 경우 대기업 대출은 기업들이 회사채 대신 대출을 통한 자금 조달을 늘리면서 전년에 이어 높은 증가세를 보였으나, 중소기업 대출은 개인사업자에 대한 코로나19 지원 대출의 상환이 늘면서 증가 폭이 축소되었다. 가계대출은 주택 매매 거래 증가 및 대출 규제 완화, 주택금융공사의 특례 보금자리론 등 정책 금융상품 공급 확대에 힘입어 예금은행 주택담보대출과 기타 금융기관 대출이 늘어나면서 증가 전환하였다. 다만, 높은 금리 수준 및 총부채 원리금 상환 비율(DSR, Debt Service Ratio) 규제, 지방 부동산 경기 회복 지연 등의 영향으로 예금은행 기타 대출과 비은행 예금 취급 기관은 감소세를 지속하였다.경제 동향 금융·외환시장 외환시장 원/달러 환율 상승, 변동성 소폭 축소 2023년 말 원/달러 환율(1,288.

# Text Splitting

In [6]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,   # 원하는 청크 크기 설정
    chunk_overlap=50,  # 중첩되는 토큰 수
    separators=["\n\n", "\n", ". ", "? ", "! "]  # 사용자 정의 분할 함수 추가
)

#text_splitter = CharacterTextSplitter(separator = "\n", chunk_size = 300, chunk_overlap  = 100, length_function = len,)

splitted_documents = text_splitter.split_documents(documents)

# 분할 후 각 문서의 내용을 정리
# 문장이 마침표로 시작하면 마침표 제거
# 문장이 마침표로 끝나지 않으면 마침표 추가
for i in range(len(splitted_documents)):
    splitted_documents[i].page_content = splitted_documents[i].page_content.strip('. ').strip()
    if not splitted_documents[i].page_content.endswith('.'):
        splitted_documents[i].page_content += '.'

# 분할 결과 출력
print(f'분할 전 문서 개수: {len(documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')



분할 전 문서 개수: 1
분할 후 문서 개수: 61


In [7]:
splitted_documents 

[Document(metadata={'source': '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023_2.txt'}, page_content='기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다.'),
 Document(metadata={'source': '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023_2.txt'}, page_content='초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하였다. 일부 업권의 건전성 우려, 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었다.'),
 Document(metadata={'source': '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023_2.txt'}, page_content='일부 업권의 건전성 우려, 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었다. 자산운용사 수신은 주가 상승, 대내외 긴축기조 완화 기대 등으로 주식형 및 채권형 펀드로 자금이 유입되면서 증가 폭이 확대되었다. 2023년 중 기업대출은 은행과 비은행 예금 취급 기관 모두 전년 대비 증가 폭이 축소되었다.'),
 Document(metadata={'source': '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023_2.txt'}, page_content='특히 비은행 예

# Vector DB

In [8]:
#분할된 문장을 벡터db에 저장
#!pip install tiktoken
#!pip install chromadb==0.5.3 
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
from langchain import FAISS
import faiss
import numpy as np

# SentenceTransformer 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="jeonseonjin/embedding_BAAI-bge-m3")

# SentenceTransformer 적재를 위해 Document 객체에서 텍스트 추출
texts = [doc.page_content for doc in splitted_documents]

# 문서 임베딩 생성
embeddings = embedding_model.embed_documents(texts)

# FAISS 인덱스 생성
d = len(embeddings[0])  # 임베딩 차원 (예: 384차원)
#index = faiss.IndexFlatL2(d)  # L2 거리 기반의 FAISS 인덱스 생성
index = faiss.IndexFlatIP(d)  # 내적을 사용하여 코사인 유사도 기반 인덱스 생성
index.add(np.array(embeddings))

# FAISS 인덱스를 파일로 저장
faiss.write_index(index, 'faiss_index.bin')

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5186/260949204.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jeonseonjin/embedding_BAAI-bge-m3")


# Retrieval

In [9]:

# 벡터db에 연결
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
from langchain import FAISS

# FAISS 인덱스를 파일에서 불러오기
if os.path.exists('faiss_index.bin'):
    index = faiss.read_index('faiss_index.bin')
    print("FAISS index loaded from file.")
else:
    print("FAISS index file not found.")

# SentenceTransformer 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="jeonseonjin/embedding_BAAI-bge-m3")

# LangChain의 FAISS와 연결
docstore = InMemoryDocstore({i: Document(page_content=texts[i]) for i in range(len(texts))})
database = FAISS(embedding_function=embedding_model, index=index, docstore=docstore, index_to_docstore_id={i: i for i in range(len(texts))})

# 쿼리 제시
query = '올해 한국은행의 정책목표 중 가장 중요한 것은 무엇인가? 답변과 함께 근거를 제시하라'
results = database.similarity_search(query, k=5) 


# 답변에서 중복 제거
unique_results = {result.page_content: result for result in results}.values()

print(f'문서 개수: {len(unique_results)}')

for d in unique_results:
    print(f'문서 내용: {d.page_content}')

FAISS index loaded from file.
문서 개수: 5
문서 내용: 한국은행은 이러한 정책 목표를 효과적으로 달성하는 방향으로 기준금리를 운영하고 있으며, 공개시장 운영, 여수신 제도, 지급준비 제도 등의 정책 수단을 활용하고 있다.
문서 내용: 통화신용정책 한국은행은 통화신용정책의 효율적인 수립과 집행을 통해 물가 안정을 도모하고 금융 안정에도 유의하면서 국민경제의 건전한 발전에 이바지하기 위해 최선의 노력을 기울이고 있다. 통화정책 운영체계로 물가 안정 목표제를 채택하고 있으며 동 체계하에서도 정책 결정은 물가 상황 외에 경제 성장, 금융 안정 등을 종합적으로 고려하여 이루어진다.
문서 내용: 한국은행이 수행한 통화신용정책의 상세한 내용은 한국은행이 발간하는 ｢통화신용정책보고서(분기)｣의 ｢Ⅱ. 통화신용정책 운영｣에서 확인할 수 있다. 한국은행의 통화정책 체계에 대한 상세한 설명은 한국은행이 발간한 ｢한국의 통화정책(2017)｣에서 확인할 수 있다.업무 현황 통화신용정책이라고 판단하였기 때문이다.
문서 내용: 가. 물가 안정 목표 한국은행은 2016년 이후 물가 안정 목표를 소비자 물가 상승률(전년 동기 대비) 기준 2%로 설정하고 있다. 2019년부터 적용되는 ｢2019년 이후 물가 안정 목표｣ 설정 시에는 목표 수준을 종전(2016∼18년)과 동일하게 2%로 유지하는 가운데, 물가 안정 목표의 적용 기간을 특정하지 않는 방식으로 변경하였다.
문서 내용: 한국은행은 이러한 대내외 정책 여건, 즉 물가 상승률이 기조적인 둔화 흐름을 이어갔지만 연중 물가 목표를 상회하는 오름세가 이어지고 경기와 금융안정 측면의 리스크가 상존하고 있었던 데다가 가계부채 추이, 미연준 통화정책, 지정학적 리스크 등과 관련한 높은 불확실성도 지속된 점을 고려할 때 기준금리를 긴축적인 수준에서 유지하고 그간의 3.0% 포인트 인상의 파급 효과를 점검하면서 추가 인상 필요성을 판단해 나가는 것이 필요하다.주요 이슈이다.


In [10]:
query = '최근 한국은행이 기준금리를 조정한 이유는 무엇인가요?'

results = database.similarity_search(query, k=5) 

# 답변에서 중복 제거
unique_results = {result.page_content: result for result in results}.values()

print(f'문서 개수: {len(unique_results)}')

for d in unique_results:
    print(f'문서 내용: {d.page_content}')

문서 개수: 5
문서 내용: 한국은행은 이러한 정책 목표를 효과적으로 달성하는 방향으로 기준금리를 운영하고 있으며, 공개시장 운영, 여수신 제도, 지급준비 제도 등의 정책 수단을 활용하고 있다.
문서 내용: 한국은행은 이러한 대내외 정책 여건, 즉 물가 상승률이 기조적인 둔화 흐름을 이어갔지만 연중 물가 목표를 상회하는 오름세가 이어지고 경기와 금융안정 측면의 리스크가 상존하고 있었던 데다가 가계부채 추이, 미연준 통화정책, 지정학적 리스크 등과 관련한 높은 불확실성도 지속된 점을 고려할 때 기준금리를 긴축적인 수준에서 유지하고 그간의 3.0% 포인트 인상의 파급 효과를 점검하면서 추가 인상 필요성을 판단해 나가는 것이 필요하다.주요 이슈이다.
문서 내용: 한국은행은 콜금리(익일물)를 금융통화위원회가 정한 기준금리 수준에서 유지하기 위해 통화안정증권, 환매조건부증권(RP), 통화안정계정 등 다양한 공개시장 운영 수단을 활용하여 유동성을 신축적으로 조절하였다. 아울러 공개시장 운영 여건 변화에 맞춰 통화안정증권 발행 제도를 개선함으로써 유동성 조절의 효율성을 제고하였다.
문서 내용: 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다.
문서 내용: 나. 기준금리 한국은행은 2023년 중 성장세를 점검하면서 중기적 시계에서 물가 상승률이 목표 수준(2%)에서 안정될 수 있도록 기준금리를 긴축적인 수준에서 운용하였다.


In [11]:
query = '한국은행의 통화정책과 금융안정 정책의 차이점은 무엇인가요?'

results = database.similarity_search(query, k=5) 

# 답변에서 중복 제거
unique_results = {result.page_content: result for result in results}.values()

print(f'문서 개수: {len(unique_results)}')

for d in unique_results:
    print(f'문서 내용: {d.page_content}')

문서 개수: 5
문서 내용: 통화신용정책 한국은행은 통화신용정책의 효율적인 수립과 집행을 통해 물가 안정을 도모하고 금융 안정에도 유의하면서 국민경제의 건전한 발전에 이바지하기 위해 최선의 노력을 기울이고 있다. 통화정책 운영체계로 물가 안정 목표제를 채택하고 있으며 동 체계하에서도 정책 결정은 물가 상황 외에 경제 성장, 금융 안정 등을 종합적으로 고려하여 이루어진다.
문서 내용: 한국은행이 수행한 통화신용정책의 상세한 내용은 한국은행이 발간하는 ｢통화신용정책보고서(분기)｣의 ｢Ⅱ. 통화신용정책 운영｣에서 확인할 수 있다. 한국은행의 통화정책 체계에 대한 상세한 설명은 한국은행이 발간한 ｢한국의 통화정책(2017)｣에서 확인할 수 있다.업무 현황 통화신용정책이라고 판단하였기 때문이다.
문서 내용: 한국은행은 통화신용정책 등을 수행함에 있어 금융기관 등을 대상으로 여수신 업무를 수행한다. 2023년 중 운용한 여신 업무로는 금융중개지원대출, 자금조정대출, 일중당좌대출 등이 있으며, 수신 업무로는 당좌예금, 자금조정예금 및 통화 안정계정 예치금 등이 있다.
문서 내용: 한국은행은 콜금리(익일물)를 금융통화위원회가 정한 기준금리 수준에서 유지하기 위해 통화안정증권, 환매조건부증권(RP), 통화안정계정 등 다양한 공개시장 운영 수단을 활용하여 유동성을 신축적으로 조절하였다. 아울러 공개시장 운영 여건 변화에 맞춰 통화안정증권 발행 제도를 개선함으로써 유동성 조절의 효율성을 제고하였다.
문서 내용: 입력: 23 Ⅰ 경제 동향 2 금융･외환시장 QR BOX QR1 통화신용정책보고서 웹페이지 QR2 경제전망보고서 웹페이지 QR3 금융안정보고서 웹페이지 QR4 지역경제보고서 웹페이지 QR5 한국은행 블로그  
출력:주요 이슈.


In [12]:
query = '한국은행의 최근 금융시장 안정화 조치는 무엇이었나요?'

results = database.similarity_search(query, k=5) 

# 답변에서 중복 제거
unique_results = {result.page_content: result for result in results}.values()

print(f'문서 개수: {len(unique_results)}')

for d in unique_results:
    print(f'문서 내용: {d.page_content}')

문서 개수: 5
문서 내용: 통화신용정책 한국은행은 통화신용정책의 효율적인 수립과 집행을 통해 물가 안정을 도모하고 금융 안정에도 유의하면서 국민경제의 건전한 발전에 이바지하기 위해 최선의 노력을 기울이고 있다. 통화정책 운영체계로 물가 안정 목표제를 채택하고 있으며 동 체계하에서도 정책 결정은 물가 상황 외에 경제 성장, 금융 안정 등을 종합적으로 고려하여 이루어진다.
문서 내용: 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다.
문서 내용: 한국은행은 이러한 정책 목표를 효과적으로 달성하는 방향으로 기준금리를 운영하고 있으며, 공개시장 운영, 여수신 제도, 지급준비 제도 등의 정책 수단을 활용하고 있다.
문서 내용: 한국은행은 통화신용정책 등을 수행함에 있어 금융기관 등을 대상으로 여수신 업무를 수행한다. 2023년 중 운용한 여신 업무로는 금융중개지원대출, 자금조정대출, 일중당좌대출 등이 있으며, 수신 업무로는 당좌예금, 자금조정예금 및 통화 안정계정 예치금 등이 있다.
문서 내용: 한국은행은 콜금리(익일물)를 금융통화위원회가 정한 기준금리 수준에서 유지하기 위해 통화안정증권, 환매조건부증권(RP), 통화안정계정 등 다양한 공개시장 운영 수단을 활용하여 유동성을 신축적으로 조절하였다. 아울러 공개시장 운영 여건 변화에 맞춰 통화안정증권 발행 제도를 개선함으로써 유동성 조절의 효율성을 제고하였다.


# Query Augmentation

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline

# MPS 디바이스 설정
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

#모델 설정
model_id = "beomi/gemma-ko-2b"
dtype = torch.bfloat16 if torch.backends.mps.is_available() else torch.float32

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=dtype  # torch_dtype 설정
)

# 모델을 MPS로 이동
model = model.to(device)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.43it/s]


In [14]:
# Hugging Face pipeline을 사용하여 LLM 구성
hf_pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_length=1024,  
    temperature=0.7, 
    top_k=50,  
    top_p=0.9,
    do_sample=True, 
    truncation=True,
    device=0 if device.type == "mps" else -1  # MPS가 있으면 0, 없으면 CPU
)

# LangChain에서 Hugging Face 모델을 LLM으로 사용
huggingface_llm = HuggingFacePipeline(pipeline=hf_pipeline)

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5186/1733727166.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  huggingface_llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [15]:
from langchain.chains import RetrievalQA

# Retriever 설정
retriever = database.as_retriever()  # 데이터베이스를 Retriever로 변환

get_answer = RetrievalQA.from_llm( #RetrievalQA를 초기화
    llm=huggingface_llm, #Chat models를 지정
    retriever = retriever, #Retriever를 지정
    return_source_documents=False #응답에 원본 문서를 포함할지 결정
)

query = "올해 한국은행의 정책목표 중 가장 중요한 것은 무엇인가?"

def answer_to_query(query):
    answer = get_answer(query)  # 답변 요청
    result_text = answer.get('result', '')  # 결과에서 'result' 키를 안전하게 가져오기

    # 특정 텍스트 패턴 제거 로직
    if 'Helpful Answer:\n' in result_text:
        parts = result_text.split('Helpful Answer:\n')
        answer_parsed = parts[1].strip() if len(parts) > 1 else parts[0].strip()
    elif 'Answer:' in result_text:
        answer_parsed = result_text.split('Answer:')[-1].strip()
    else:
        answer_parsed = result_text.strip()  # 기본적으로 텍스트 전체를 반환

    return answer_parsed

answer_to_query(query)

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5186/3408086933.py:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = get_answer(query)  # 답변 요청
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'기준금리 인상 필요성을 판단하는 것이 가장 중요하다.'

In [16]:
answer_to_query('최근 한국은행이 기준금리를 조정한 이유는 무엇인가요?')

KeyboardInterrupt: 

In [17]:
import gradio as gr
from langchain.chains import RetrievalQA

# Retriever 설정 (database 및 huggingface_llm은 미리 정의되어 있어야 함)
retriever = database.as_retriever()  # 데이터베이스를 Retriever로 변환

# RetrievalQA 초기화
get_answer = RetrievalQA.from_llm(
    llm=huggingface_llm,  # Chat models를 지정
    retriever=retriever,  # Retriever를 지정
    return_source_documents=False  # 응답에 원본 문서를 포함할지 결정
)

# 질문에 대한 답변 생성 함수 (RetrievalQA 활용)
def chat_with_retrieval(message, history):
    # 사용자의 질문에 대해 RetrievalQA를 사용하여 답변 생성
    response = get_answer(message)
    result_text = response.get('result', '')

    # 리스트일 경우 이를 하나의 문자열로 변환
    if isinstance(result_text, list):
        result_text = ' '.join([str(item) for item in result_text])

    # 특정 텍스트 패턴 제거 로직
    if 'Helpful Answer:\n' in result_text:
        parts = result_text.split('Helpful Answer:\n')
        answer_parsed = parts[1].strip() if len(parts) > 1 else parts[0].strip()
    elif 'Answer:' in result_text:
        answer_parsed = result_text.split('Answer:')[-1].strip()
    else:
        answer_parsed = result_text.strip()


    # 불필요한 패턴 (공백 + 문자 + :) 제거. 해당 패턴 이후 내용 제거
    # 정규 표현식으로 공백 뒤에 [문자]: 형태를 탐색하여 그 뒤를 제거
    answer_parsed  = re.split(r"\s\w+:", answer_parsed)[0].strip()
    
    # 질문과 답변을 히스토리에 저장 (history는 대화 히스토리)
    history.append((message, answer_parsed))  
    
    # Gradio가 (응답, history)를 반환해야 하므로, 대화 기록과 함께 반환
    return history, history

# Gradio Chatbot 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()  # 대화 기록을 표시하는 컴포넌트
    msg = gr.Textbox(label="질문 입력")  # 질문 입력을 위한 텍스트 박스
    clear = gr.Button("대화 기록 초기화")  # 대화 기록 초기화 버튼

    # 대화가 시작될 때 실행할 동작 정의
    msg.submit(chat_with_retrieval, inputs=[msg, chatbot], outputs=[chatbot, chatbot])

    # 기록 초기화 버튼 동작 정의
    clear.click(lambda: None, None, chatbot, queue=False)

# 앱 실행
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://0f9e8f92e0595fe80d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Matplotlib created a temporary cache directory at /var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/matplotlib-79i_5ymi because the default path (/Users/jaesolshin/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [ ]:
from langchain.prompts import PromptTemplate
query = "2023년 통화정책의 가장 중요한 목표와 고려사항은?"

results = database.similarity_search(query, k=5) 

context = "Relevant documents:\n"  #조회를 통해 얻은 정보를 저장할 변수 초기화

for i, result in enumerate(results):
    context += f"""
-----------------------------------
Doc {i+1}:
{result.page_content}
"""

prompt = PromptTemplate(
    template="""경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 하나의 완결된 문장으로 작성. 중간에 끊기지 않도록 문장을 완성할 것. 2) 필요한 정보를 모두 담도록 할 것. 3) 출처를 명확히 제시할 것. 4) 개조식으로 작성하지 말 것

***Information***
{document}

***Question***
{query} 


***Answer***""",
    input_variables=['document', 'query']
)

prompt = prompt.format(document=context, query=query)
print(prompt)

경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 하나의 완결된 문장으로 작성. 중간에 끊기지 않도록 문장을 완성할 것. 2) 필요한 정보를 모두 담도록 할 것. 3) 출처를 명확히 제시할 것. 4) 개조식으로 작성하지 말 것

***Information***
Relevant documents:

-----------------------------------
Doc 1:
이는 2023년 중 성장률이 2022년보다 낮아질 것으로 예상되었지만, 물가 상승률이 여전히 5%대의 높은 수준을 나타내고 있고 앞으로도 상당 기간 목표 수준을 상회할 것으로 전망됨에 따라 물가 안정을 위해서는 통화 긴축 강도를 강화하는 것이 적절하다고 판단하였기 때문이다.

-----------------------------------
Doc 2:
한국은행은 통화신용정책 등을 수행함에 있어 금융기관 등을 대상으로 여수신 업무를 수행한다. 2023년 중 운용한 여신 업무로는 금융중개지원대출, 자금조정대출, 일중당좌대출 등이 있으며, 수신 업무로는 당좌예금, 자금조정예금 및 통화 안정계정 예치금 등이 있다.

-----------------------------------
Doc 3:
특히 연간 변동폭(연중 최고치-연중 최저치) 기준으로는 2023년 중 143원을 기록하여 2022년(253원)의 절반 수준으로 크게 축소되었다.외화자금시장 안정세 유지 2023년 중 외화자금시장은 양호한 외화유동성 사정이 지속되며 대체로 안정세를 유지하였다.

-----------------------------------
Doc 4:
한국은행은 중기적 시계에서 소비자 물가 상승률이 물가 안정 목표에 근접하도록 통화신용정책을 운영하고 있다. 2023년 중 소비자 물가 상승률은 국제 원자재 가격 하락, 통화 긴축에 따른 수요 측 물가 압력 완화 등의 영향으로 3.6%로 낮아졌으나 2022년에 이어 물가 안정 목표를 상회하였다.

-----------------------------------
D

In [ ]:
import gradio as gr
import re
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿 설정
prompt_template = PromptTemplate(
    template="""경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 144자 정도의 완결된 문장으로 작성할 것 2) 개조식으로 작성하지 말 것 3) 한국은행이나 경제에 관련되지 않은 질문에 대한 답변은 거절할 것.

***Information***
{context}

***Question***
{query}

***Answer***""",
    input_variables=['context', 'query']
)


# 질문에 대한 답변 생성 함수 (RetrievalQA 활용)
def chat_with_retrieval(message, history):

    # 유사도가 높은 문장을 검색
    results = database.similarity_search(message, k=5) 

    #조회를 통해 얻은 정보를 저장
    context = "Relevant documents:\n"  

    for i, result in enumerate(results):
        context += f"""
        -----------------------------------
        Doc {i+1}:
        {result.page_content}
        """

    # 프롬프트 템플릿 적용
    prompt = prompt_template.format(context=context, query=message)
    
    # LLM을 통해 답변 생성
    llm_response = huggingface_llm(prompt)  # 여기서는 LLM 호출 결과가 문자열로 반환됩니다.
    result_text = llm_response  # 이미 문자열이므로 `get` 대신 바로 result_text로 사용

    # 불필요한 패턴 제거 로직
    if '***Answer***' in result_text: 
        answer_parsed = result_text.split('***Answer***')[-1].strip()
    elif '***information***' in result_text:
        answer_parsed = result_text.split('***Question***')[-1].strip()
    elif '***Answer***' in result_text:
        answer_parsed = result_text.split('***Answer***')[-1].strip()        
    else:
        answer_parsed = result_text.strip()

    # 불필요한 패턴 (공백 + 문자 + :) 제거
    answer_parsed = re.sub(r'\s\w+:', '', answer_parsed).strip()

    # 질문과 답변을 히스토리에 저장 (history는 대화 히스토리)
    history.append((message, answer_parsed))

    # Gradio가 (응답, history)를 반환해야 하므로, 대화 기록과 함께 반환
    return history, history

# Gradio Chatbot 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()  # 대화 기록을 표시하는 컴포넌트
    msg = gr.Textbox(label="질문 입력")  # 질문 입력을 위한 텍스트 박스
    clear = gr.Button("대화 기록 초기화")  # 대화 기록 초기화 버튼

    # 대화가 시작될 때 실행할 동작 정의
    msg.submit(chat_with_retrieval, inputs=[msg, chatbot], outputs=[chatbot, chatbot])

    # 기록 초기화 버튼 동작 정의
    clear.click(lambda: None, None, chatbot, queue=False)

# 앱 실행
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7867
Running on public URL: https://7a32de75ce728b4edc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Matplotlib created a temporary cache directory at /var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/matplotlib-cjamy08h because the default path (/Users/jaesolshin/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_13125/2916617150.py:41: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_response = huggingface_llm(prompt)  # 여기서는 LLM 호출 결과가 문자열로 반환됩니다.
